In [2]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import pandas as pd
from google.colab import drive
import os
import time
from tqdm import tqdm
import math
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.image as mpimg
from natsort import natsorted
import zipfile
from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Opening and Visualizing Files

In [3]:
### TO DO
# Figure out how to automate importing files from Google Drive and opening them into a pd.DataFrame

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# Open WAV file samples and sampling rate (put in loop to extract files as new variables)

# Pre-processing here (equalizer function)

# Add variables to dataloader



# Folder with multiple files
dir = '/content/drive/Shareddrives/BE5740/Dataset/All files/'

# Initialize full_info list for data to turn into DataFrame
full_info = []

# Iterate over all files within main folder
for main_folder in tqdm(os.listdir(dir)):
  set_folder = dir + str(main_folder)   # Extract name of the set of trials folder

  # Iterate over folder of set of trials
  for folder in tqdm(os.listdir(set_folder)):
    ind_folder = str(set_folder) + '/' + str(folder)   # Extract name of the individual folder

    # Iterate over all files in single trial folder
    for file in os.listdir(ind_folder):

      # Determine if it's an audio file
      if file.endswith(".wav"):

        sample_rate, samples = wavfile.read(ind_folder + '/' + str(file)) # read file and extract info

        # Set variables for characteristics of the spectrogram
        frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

        trial_info = {'Trial': str(file)[0:-4], 'Frequency': frequencies, 'Raw Data': samples}

        full_info.append(trial_info)


df = pd.DataFrame(full_info)

  6%|▌         | 3/52 [00:01<00:28,  1.72it/s]

In [1]:
df

NameError: name 'df' is not defined

In [ ]:
# Example Using One File
file_name = '/content/drive/Shareddrives/BE5740/Dataset/All files/641-718/718_P/718_AUDIO.wav'
sample_rate, samples = wavfile.read(file_name)

# Set variables for characteristics of the spectrogram
frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

In [ ]:
# Testing out different filters and validating with spectrogram (find way to automate putting variables through filter and saving as new
# variables))
# We should figure out which frequencies human voices are generally at and filter everything outside of that out (AKA none of these
# filters are doing much and I don't want to filter out the actual signal)

### TO DO

# Low pass butterworth filter
b, a = signal.butter(3, 0.05)
zi = signal.lfilter_zi(b, a)
z, _ = signal.lfilter(b, a, samples, zi=zi*samples[0])
z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
y_lp_bw = signal.filtfilt(b, a, samples)

# Notch filter
fs = sample_rate  # Sample frequency (Hz)
f0 = 10  # Frequency to be removed from signal (Hz)
Q = 30.0  # Quality factor
b, a = signal.iirnotch(f0, Q, fs)
y_iirnotch = signal.filtfilt(b, a, samples)

In [ ]:
# Plot the spectrogram data to visualize
plt.specgram(samples[int(1e6):int(1e6) + sample_rate * 2])
cbar = plt.colorbar(label = 'Amplitude (dB)')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title(f'Spectrogram of {file_name[:-4]} file')
plt.show()

In [ ]:
# Plot the spectrogram with different filters
# COMMENT HERE WHICH FILTER IS BEST
filter = 'Lowpass Butterworth Filter'
plt.specgram(y_lp_bw[int(1e6):int(1e6) + sample_rate * 2])
cbar = plt.colorbar(label = 'Amplitude (dB)')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title(f'Spectrogram of {file_name[-13:-4]} file with {filter} applied')
plt.show()

In [ ]:
plt.magnitude_spectrum(samples)
plt.title(f'Magnitude Spectrum of {file_name[:-4]} file')

# Open, Visualize, and Create pd.DataFrames out of CSV Files Here

In [ ]:
# Change path here for file
file_path = '/content/drive/Shareddrives/BE5740/Dataset/All files/461-640/462/462_OpenFace2.1.0_Pose_gaze_AUs.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
# 30 samples is 1 second so iterate over 120 samples for 4 seconds
four_sec = 120

expressions = df[df.columns[18:-1]]

for i in range(math.ceil(len(df) / 120)):
  temp = expressions[i * four_sec:(i+1) * four_sec]
  col_mean = pd.DataFrame(temp.mean()).T
  col = col_mean.max().idxmax()
  print(col)

In [ ]:
## Automate looping through CSV files and creating ground truth "happy" or "sad" score based on average facial expression
## over 4 seconds in each video. Use this in conjunction with 4 seconds of spectrogram information to train the neural net

# Create Pipeline Here

# Make Predictions Here

In [ ]:
train_image_ids = []   # These are the image ids of the images that we will be using for training
val_image_ids = []     # These are the image ids of the images that we will be using for validation
test_image_ids = []    # These are the image ids of the images that we will be using for testing

# train_image_ids

while(1):
    ran_num = np.random.randint(low = 0, high = 1803460) # we should change the "high"
                                          # number to the amount of samples we have for indexing purposes
    if ran_num not in train_image_ids:
        train_image_ids.append(ran_num)
    if len(train_image_ids) == 10:   # Can change this number to how many images we want in training
        break

# val_image_ids

while(1):
    ran_num = np.random.randint(low = 0, high = 1803460)
    if ran_num not in train_image_ids and ran_num not in val_image_ids:
        val_image_ids.append(ran_num)
    if len(val_image_ids) == 10:   # Can change this number to how many images we want in val
        break

# test_imge_ids

while(1):
    ran_num = np.random.randint(low = 0, high = 1803460)
    if ran_num not in train_image_ids and ran_num not in val_image_ids and ran_num not in test_image_ids:
        test_image_ids.append(ran_num)
    if len(test_image_ids) == 5:   # Can change this number to how many images we want in testing
        break

train_image_ids = torch.tensor(train_image_ids)  # Converting train_image_ids list to a tensor
val_image_ids = torch.tensor(val_image_ids)      # Converting val_image_ids list to a tensor
test_image_ids = torch.tensor(test_image_ids)    # Converting test_image_ids list to a tensor


train_images = torch.utils.data.Subset(dataset=data, indices=train_image_ids)    # Getting subset of images using these 5000 train_image_ids
val_images = torch.utils.data.Subset(dataset=data, indices=val_image_ids)        # Getting subset of images using these 500 val_image_ids
test_images = torch.utils.data.Subset(dataset=data, indices=test_image_ids)      # Getting subset of images using these 500 test_image_ids

# Starting DataLoader and CNN Here

In [ ]:
pic = '/content/drive/Shareddrives/BE5740/Dataset/spectrograms/380_seg_489.png'
img = mpimg.imread(pic)
#plt.imshow(img)
#plt.show()
img

In [ ]:
spec_dir = '/content/drive/Shareddrives/BE5740/Dataset/spectrograms/'

# Initialize full_info list for data to turn into DataFrame
spectrograms = {'Spectrogram': [], 'Label': []}

# Iterate over all spectrograms within spectrogram folder
for spectrogram in tqdm(os.listdir(spec_dir)):
  spectrograms['Spectrogram'].append(spectrogram)


spectrograms['Spectrogram'] = natsorted(spectrograms['Spectrogram'])

In [ ]:
mood_dir = '/content/drive/Shareddrives/BE5740/Dataset/mood/'

csv = []

for moods in tqdm(os.listdir(mood_dir)):
  csv.append(moods)
  #spectrograms['Label'].append(pd.read_csv(moods))

csv = natsorted(csv)

for val in tqdm(csv):
  moods = pd.read_csv(mood_dir + val)
  for index, row in moods.iterrows():
    if row['filename'][0:-4] + '.png' in spectrograms['Spectrogram']:
      spectrograms['Label'].append(row['label'])

In [ ]:
zip_dir = '/content/drive/Shareddrives/BE5740/Dataset/spectrograms_zip/spectrograms.zip'

zip = zipfile.ZipFile(zip_dir)
images = natsorted(zip.namelist())

imgzip = zipfile.ZipFile(zip_dir)
inflist = imgzip.infolist()

ifile = imgzip.open(images[0])
img = Image.open(ifile)
display(img)

In [ ]:
input_data = []

zip_dir = '/content/drive/Shareddrives/BE5740/Dataset/spectrograms_zip/spectrograms.zip'

zip = zipfile.ZipFile(zip_dir)
images = natsorted(zip.namelist())

imgzip = zipfile.ZipFile(zip_dir)
inflist = imgzip.infolist()

transform = transforms.Compose([
    transforms.PILToTensor()
])


for pic in tqdm(images):
  ifile = imgzip.open(pic)
  img = Image.open(ifile)
  img_tensor = transform(img)
  input_data.append(img_tensor)

In [ ]:
input_data = np.array(input_data)
input_data = torch.tensor(input_data)
input_data = input_data.float()

In [ ]:
target_data = []

for val in spectrograms['Label']:
  target_data.append(val)

In [ ]:
target_data = torch.tensor(target_data)
target_data = target_data.float()

In [ ]:
train_input = input_data[0:2000]
train_target = target_data[0:2000]

test_input = input_data[2000:]
test_target = target_data[2000:]

In [ ]:
train_dataset = TensorDataset(train_input, train_target)
test_dataset = TensorDataset(test_input, test_target)

train_dataloader = DataLoader(train_dataset,batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset,batch_size=1, shuffle=False)

In [ ]:
# Define CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32768, 64)
        self.fc2 = nn.Linear(64, 2)  # 2 output classes (binary classification)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 32768)  # Flatten the feature maps
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Instantiate model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    model.train()
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        inputs = inputs.to(torch.long)
        targets = targets.to(torch.long)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

In [ ]:
# Evaluation
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for inputs, targets in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        total_samples += targets.size(0)

accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy}')